In [20]:
import string

exclude = set(string.punctuation)

def strip_punct(text):
    s = ''.join(ch for ch in text if ch not in exclude)
    return s

In [38]:
import nltk

def check_stop(text,stopwords):
    text = text.split(" ")
    for w in text:
        if w not in stopwords:
            return False
    return True


sentence = "Obama won the 2008 Iowa caucuses while Hillary was leading in the polls."
ngram_up_to = 3

def get_all_ngrams(text,ngram_up_to):
    """Returns all ngrams from a text up to a certain number.
    
    Args:
        text: a string.
        ngram_up_to: a integer.
    Returns:
        A list of ngrams.
    """
    
    tokens = nltk.word_tokenize(text)
    ngrams = [" ".join(x) for n in range(1,ngram_up_to+1) for x in nltk.ngrams(tokens,n)]        
    return ngrams
 
ngrams = get_all_ngrams(sentence,ngram_up_to)
print (ngrams[:20])

def remove_substrings(candidate_mentions,stopwords,cutoff):
    """Removes very short candidate mentions (<2 chars) 
    and ngrams that are substrings of a more popular one.
    
    Args:
        candidate_mentions: list of tuples (candidate_mention (str), probability score (float)).
    Returns:
        A final list of mentions as strings.      
    """
    candidate_mentions = Counter(candidate_mentions).most_common()
    
    stopwords = [x for x,y in candidate_mentions if len(x.split(" "))<2][:stopwords]

    candidate_mentions = [(cand,score) for cand,score in candidate_mentions if len(cand)>3 and cand not in stopwords][:cutoff]
    
    one_word_cand_mentions = [(cand,score) for cand,score in candidate_mentions if len(cand.split(" "))<2]

    longer_cand_mentions = [(cand,score) for cand,score in candidate_mentions if (cand,score) not in one_word_cand_mentions and check_stop(cand,stopwords) is False]
        
    longer_cand_mentions.sort(key=lambda cand: cand[1], reverse=True)
    one_word_cand_mentions.sort(key=lambda cand: cand[1], reverse=True)
    
    resorted_cand_mentions = longer_cand_mentions + one_word_cand_mentions
    
    mentions = []
    for cand,score in resorted_cand_mentions:
        if not any([cand in mention for mention in mentions]):
            mentions.append(cand)
    return mentions
     
mentions = remove_substrings(ngrams,stopwords=10,cutoff=1000)

print (mentions)

['Obama', 'won', 'the', '2008', 'Iowa', 'caucuses', 'while', 'Hillary', 'was', 'leading', 'in', 'the', 'polls', '.', 'Obama won', 'won the', 'the 2008', '2008 Iowa', 'Iowa caucuses', 'caucuses while']
['leading in', 'in the', 'the polls', 'polls .', 'was leading in', 'leading in the', 'in the polls', 'the polls .']


In [2]:
from bs4 import BeautifulSoup
import requests


def Wikipedia_search(text,lang):
    url = "https://"+lang+".wikipedia.org/w/index.php?cirrusUserTesting=control&search="+text+"&title=Sp%C3%A9cial%3ARecherche&fulltext=1&ns0=1"
    
    response = requests.get(url)
    data = response.text
    soup = BeautifulSoup(data)
    try:
        tabledata = soup.find("ul", attrs={"class": "mw-search-results"})
        first_res = tabledata.findAll("li", attrs={"class": "mw-search-result"})[0]
        first_entity = first_res.find("a")["href"]
        return first_entity
    except AttributeError:
        return None
Wikipedia_search("INPI","fr")

'/wiki/INPI'

In [3]:
import spacy

In [3]:
import pickle
import pandas as pd

langs_dict = {"ger":"de","fre":"fr","it":"it","en":"en","sl":"sl","pl":"pl","fi":"fi"}

with open('larger_selected_dataset.pickle', 'rb') as f:

#with open('selected_dataset.pickle', 'rb') as f:
    df = pickle.load(f)
    
df['langMaterial'] = df['langMaterial'].map(langs_dict)


In [4]:
df.head()

,id,langMaterial,unitTitle,titleProper,scopeContent,topic,filename
0,C122304194,fr,Usine Giraud,"119 J - Arnoux, fabrique de tracteurs, Miramas...",Plan des ateliers de construction V. Giraud fi...,[economics],economics.json
1,C122304196,fr,Documents généraux.,"119 J - Arnoux, fabrique de tracteurs, Miramas...",Historique par monsieur Hervé Arnoux (1993). C...,[economics],economics.json
2,C122304197,fr,Réparations et représentations pour les automo...,"119 J - Arnoux, fabrique de tracteurs, Miramas...","Garage Arnoux : vue de la façade sud, le long ...",[economics],economics.json
3,C122304198,fr,Motoculteurs Arnoux.,"119 J - Arnoux, fabrique de tracteurs, Miramas...",Brevet d'invention pour un petit tracteur moto...,[economics],economics.json
4,C122304200,fr,Tracteurs Arnoux et leur outillage.,"119 J - Arnoux, fabrique de tracteurs, Miramas...",Tracteurs type VM 10 et VM 15 : prospectus et ...,[economics],economics.json


In [6]:
#de_model = spacy.load("de_core_news_sm")
#fr_model = spacy.load("fr_core_news_sm")
#en_model = spacy.load("en_core_web_sm")
#it_model = spacy.load("it_core_news_sm")

In [7]:
#model_dict = {"fr":fr_model,"en":en_model,"de":de_model,"it":it_model}

[('de', 55077), ('fr', 10581), ('en', 13), ('it', 6)]

In [5]:
topics = set([row["filename"] for index, row in df.iterrows()])

In [21]:
ngram_dict = {x:[] for x in topics}

for index, row in df.iterrows():
    lang = row["langMaterial"]
    #model = model_dict[lang]
    text = row["unitTitle"] +" "+ row["titleProper"]+" "+ row["scopeContent"]
    text = strip_punct(text)
    topic = row["filename"]
    ngram_dict[topic] += get_all_ngrams(text,3)


In [43]:
from collections import Counter

for t, ngrams in ngram_dict.items():
    print (t)
    selected = remove_substrings(ngrams,stopwords=150,cutoff=1000)
    print (selected)
    print ( " ")

catholicism.json
['été effectué', 'a été effectué', '’ église', 'l ’ église', 'JeanMarie Intérieur', 'JeanMarie Intérieur Direction', 'Grade abbé', '19041998F977133 Grade abbé', 'DE LA', 'la mosquée', 'mosquée de', 'la mosquée de', 'mosquée de Paris', 'de Versailles', 'Introduction Ce', 'effectué à', 'de reconnaissance', 'reconnaissance légale', 'Introduction Ce versement', 'été effectué à', 'effectué à la', 'demande de reconnaissance', 'de reconnaissance légale', 'père Lebret', 'ont été', 'partie du', 'loi n°', 'mondiales de', 'la jeunesse', 'la ville', 'mondiales de la', 'de la jeunesse', 'Eugène Intérieur', 'Eugène Intérieur Direction', 'Félix Intérieur', 'Félix Intérieur Direction', 'lacune Intérieur', 'abbé Fonction', 'Fonction vicaire', 'lacune Intérieur Direction', 'Grade abbé Fonction', 'Emile Intérieur', 'Emile Intérieur Direction', 'Auguste Intérieur', 'Auguste Intérieur Direction', 'Michel Intérieur', 'Michel Intérieur Direction', 'Albert Intérieur', 'Albert Intérieur Direct

In [6]:
words_dict = {}

word_langs = {}

for index, row in df.iterrows():
    lang = row["langMaterial"]
    #model = model_dict[lang]
    text = row["unitTitle"] +" "+ row["titleProper"]+" "+ row["scopeContent"]
    ngrams = get_all_ngrams(text,3)
    print (ngrams)
    break
    topic = row["filename"]
    processed_text = model(text)
    # Analyze syntax
    words = [strip_punct(chunk.text) for chunk in processed_text.noun_chunks if len(chunk.text.split(" "))<4 and len(chunk.text)>2]
    
    for entity in processed_text.ents:
        e = strip_punct(entity.text)
        if len(e)>2:
            words.append(e)
    
    for word in words:
        if word not in words_dict:
            words_dict[word] = {t:0.0 for t in topics}
            words_dict[word][topic] = 1
        else:
            words_dict[word][topic] += 1
            
        if word not in word_langs:
            word_langs[word] = {lang}
        else:
            word_langs[word].add(lang)


['Usine', 'Giraud', '119', 'J', '-', 'Arnoux', ',', 'fabrique', 'de', 'tracteurs', ',', 'Miramas', '(', '1914-1993', ')', ':', 'F684628', 'Plan', 'des', 'ateliers', 'de', 'construction', 'V.', 'Giraud', 'fils', 'aîné', '(', 'avant', '1914', '?', ')', ',', 'plan', "d'une", 'charrue', '``', 'vigneron', "''", 'modèle', '1928', ',', 'nomenclature', 'des', 'pièces', 'de', 'charrues', 'fournies', 'par', 'les', 'aciéries', 'du', 'Rhône', '[', 'sans', 'date', ']', 'et', 'les', 'aciéries', 'et', 'fonderies', 'de', "l'est", '(', '1934', ')', '.', 'Usine Giraud', 'Giraud 119', '119 J', 'J -', '- Arnoux', 'Arnoux ,', ', fabrique', 'fabrique de', 'de tracteurs', 'tracteurs ,', ', Miramas', 'Miramas (', '( 1914-1993', '1914-1993 )', ') :', ': F684628', 'F684628 Plan', 'Plan des', 'des ateliers', 'ateliers de', 'de construction', 'construction V.', 'V. Giraud', 'Giraud fils', 'fils aîné', 'aîné (', '( avant', 'avant 1914', '1914 ?', '? )', ') ,', ', plan', "plan d'une", "d'une charrue", 'charrue ``',

In [11]:
print (index)

65676


In [12]:
# this is simply for comparing across topics
ranking = {t:[] for t in topics}

for topic in topics:
    for word in words_dict.keys():
        if words_dict[word][topic]>0:
            score = words_dict[word][topic]
            ranking[topic].append([word,score])

In [13]:
for topic,rank in ranking.items():
    print (topic)
    rank.sort(key=lambda x: x[1],reverse= True)
    for word,freq in rank[:50]:
        langs = word_langs[word]
        ents = [[Wikipedia_search(word,l),l] for l in langs]
        print (word,freq,ents)
    print (" ")

slavery.json
Saint 657.0 [['/wiki/Saint', 'fr']]
S13 557 [['/wiki/Nissan_200SX', 'fr']]
Domingue 498.0 [['/wiki/Saint-Domingue_(colonie_fran%C3%A7aise)', 'fr']]
FR CAOM 363 [['/wiki/%C3%8Eles_Sous-le-Vent_(Polyn%C3%A9sie)', 'fr']]
Noirs 349 [['/wiki/Noir_(humain)', 'fr']]
Martinique 203.0 [['/wiki/Martinique', 'de'], ['/wiki/Martinique', 'fr']]
Afrique 158.0 [['/wiki/Afrique', 'fr']]
CAOM  COL 129 [['/wiki/%C3%8Eles_Sous-le-Vent_(Polyn%C3%A9sie)', 'fr']]
Guadeloupe 127.0 [['/wiki/Guadeloupe', 'de'], ['/wiki/Guadeloupe', 'fr']]
CAOM 123 [['/wiki/Archives_nationales_d%27outre-mer', 'fr']]
Guinée 118.0 [['/wiki/Guin%C3%A9e', 'fr']]
COL 117 [['/wiki/COL', 'fr']]
Guyane 115.0 [['/wiki/Guyane', 'fr']]
État 105.0 [['/wiki/%C3%89tat', 'fr']]
Bordeaux 89.0 [['/wiki/Bordeaux', 'de'], ['/wiki/Bordeaux', 'fr']]
Antilles 89.0 [['/wiki/Antilles', 'fr']]
Sénégal 84.0 [['/wiki/S%C3%A9n%C3%A9gal', 'fr']]
Amérique 84.0 [['/wiki/Am%C3%A9rique', 'fr']]
Nantes 81.0 [['/wiki/Nantes', 'fr']]
Sainte 74.0 [['/

In [14]:
# this is simply for comparing across topics
ranking = {t:[] for t in topics}

for topic in topics:
    for word in words_dict.keys():
        if words_dict[word][topic]>0:
            mean = [y for x,y in words_dict[word].items() if x!=topic]
            mean = sum(mean)/len(mean)
            if mean != 0:
                score = words_dict[word][topic] - mean
                ranking[topic].append([word,score])

In [15]:
for topic,rank in ranking.items():
    print (topic)
    rank.sort(key=lambda x: x[1],reverse= True)
    for el in rank[:50]:
        print (el)
    print (" ")

slavery.json
['Saint', 627.75]
['Domingue', 497.75]
['Martinique', 196.75]
['Afrique', 131.25]
['Guadeloupe', 120.5]
['Guinée', 113.5]
['Guyane', 106.25]
['État', 96.75]
['Bordeaux', 85.25]
['Antilles', 85.0]
['Amérique', 82.0]
['Nantes', 79.25]
['Sénégal', 75.25]
['Sainte', 69.5]
['Guide', 68.75]
['Cap', 57.75]
['Louis', 56.75]
['Papiers', 47.0]
['États', 46.5]
['fol', 38.0]
['Compagnie', 36.0]
['Angleterre', 35.75]
['Anglais', 35.5]
['Marine', 33.0]
['Correspondance', 32.5]
['île de France', 31.75]
['Bourbon', 30.75]
['dont', 29.75]
['F12', 28.0]
['Marie', 26.0]
['Terre', 25.0]
['de France', 23.25]
['Madagascar', 22.5]
['la France', 21.75]
['Procès', 21.0]
['Mémoire', 19.75]
['du Havre', 19.25]
['Port', 18.75]
['ministère des Affaires étrangères', 18.75]
['Malo', 18.75]
['Basse', 18.25]
['Surinam', 17.75]
['Fonds', 17.75]
['de Saint', 17.25]
['Ancien Régime', 17.25]
['Rivière', 16.75]
['Brésil', 16.5]
['Convention', 15.25]
['la Réunion', 15.0]
['Anne', 15.0]
 
catholicism.json
['Bure